# Machine Learning to predict Down Jones Industrial Average

This simple Machine Learning example shows how to predict [^DJI value](https://finance.yahoo.com/quote/%5EDJI?p=^DJI&.tsrc=fin-srch) based on the past calculated averages.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1">Setup</a></span></li><li><span><a href="#Read-data-into-a-SFrame" data-toc-modified-id="Read-data-into-a-SFrame-2">Read data into a SFrame</a></span><ul class="toc-item"><li><span><a href="#TODO:-Value-should-be-original-value" data-toc-modified-id="TODO:-Value-should-be-original-value-2.1">TODO: Value should be original value</a></span></li></ul></li><li><span><a href="#Select-the-data-to-train-and-test" data-toc-modified-id="Select-the-data-to-train-and-test-3">Select the data to train and test</a></span><ul class="toc-item"><li><span><a href="#TODO:-Let's-NOT-take-last-few-days" data-toc-modified-id="TODO:-Let's-NOT-take-last-few-days-3.1">TODO: Let's NOT take last few days</a></span></li></ul></li><li><span><a href="#Create-the-model" data-toc-modified-id="Create-the-model-4">Create the model</a></span><ul class="toc-item"><li><span><a href="#Print-example-predictions" data-toc-modified-id="Print-example-predictions-4.1">Print example predictions</a></span></li></ul></li><li><span><a href="#&quot;Be-Less-Wrong&quot;" data-toc-modified-id="&quot;Be-Less-Wrong&quot;-5">"Be Less Wrong"</a></span><ul class="toc-item"><li><span><a href="#TODO:-find-the-best-model" data-toc-modified-id="TODO:-find-the-best-model-5.1">TODO: find the best model</a></span></li></ul></li><li><span><a href="#Save-the-model" data-toc-modified-id="Save-the-model-6">Save the model</a></span></li></ul></div>

## Setup

In [1]:
# Install TuriCreate. Last updated November 4, 2020

# !pip install --upgrade pip
# !pip install Turicreate

In [2]:
import turicreate as tc

In [3]:
# Location of the spreadsheet (Comma Delimited Value) with ^DJI info that I prpared in a separate notebook.
data_path="./DATA/processed/^DJI.csv"

## Read data into a SFrame

In [4]:
# Load the data
data =  tc.SFrame(data_path)
data[363:370] # show data sample

Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/DATA/processed/^DJI.csv

Parsing completed. Parsed 100 lines in 0.035743 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,float,float,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/DATA/processed/^DJI.csv

Parsing completed. Parsed 13066 lines in 0.029967 secs.

Day,Date,Value,Original,Avg005,Avg030,Avg090,Avg180,Avg365
725033,1986-01-27,-125.0,1548.170044,-125.2,-125.1,-125.66,-126.33,0.0
725034,1986-01-28,-125.0,1561.349976,-125.0,-125.1,-125.63,-126.32,-126.98
725035,1986-01-29,-125.0,1578.099976,-125.0,-125.1,-125.61,-126.31,-126.97
725036,1986-01-30,-125.0,1572.589966,-125.0,-125.1,-125.59,-126.29,-126.96
725037,1986-01-31,-125.0,1582.910034,-125.0,-125.1,-125.57,-126.28,-126.96
725038,1986-02-01,-125.0,1582.910034,-125.0,-125.1,-125.54,-126.27,-126.95
725039,1986-02-02,-125.0,1582.910034,-125.0,-125.1,-125.52,-126.26,-126.94


### TODO: Value should be original value

Please note the the "High" is normalized to Int8, 
but for the prediciton purposes it should be an original "real" value.

## Select the data to train and test

In [5]:
row_count = len(data)
# Do not take initial year data as averages are not complete
data = data[365:row_count] 
# Make a train-test split
train_data, test_data = data.random_split(0.8)

### TODO: Let's NOT take last few days

I need to save the last few days to see if I can really predict upcoming values.

## Create the model

- https://apple.github.io/turicreate/docs/api/generated/turicreate.regression.create.html
- Automatically picks the right model based on your data.
- target: is the number to be predicted.
- features: are the the values that we ues to try to find pattern leading to prediciton.

In [6]:
model = tc.regression.create(
    train_data, 
    target='Original',
    features = [
        'Avg005',
        'Avg030',
        'Avg090',
        'Avg180',
        'Avg365'
    ],
    validation_set='auto', 
    verbose=True
)

Automatically generating validation set from 5% of the data.

Linear regression:

--------------------------------------------------------

Number of examples          : 9654

Number of features          : 5

...

| 3         | 0.022818     | 11382.255859       | 11155.746094         | 4335.525879                     | 4415.639160                       |

| 4         | 0.027737     | 8310.158203        | 8083.648438          | 3041.178467                     | 3090.190186                       |

| 5         | 0.033973     | 6048.958984        | 5822.449219          | 2134.148682                     | 2161.316406                       |

| 10        | 0.062709     | 1869.667969        | 1383.025391          | 380.208771                      | 380.107178                        |

+-----------+--------------+--------------------+----------------------+---------------------------------+-----------------------------------+

# Predict values on data that was NOT used in training

In [7]:
#test_data.explore()
test_data

Day,Date,Value,Original,Avg005,Avg030,Avg090,Avg180,Avg365
725035,1986-01-29,-125.0,1578.099976,-125.0,-125.1,-125.61,-126.31,-126.97
725040,1986-02-03,-125.0,1600.109985,-125.0,-125.1,-125.5,-126.25,-126.93
725041,1986-02-04,-125.0,1611.469971,-125.0,-125.1,-125.48,-126.24,-126.92
725044,1986-02-07,-125.0,1622.819946,-125.0,-125.1,-125.41,-126.21,-126.9
725045,1986-02-08,-125.0,1622.819946,-125.0,-125.1,-125.39,-126.19,-126.89
725046,1986-02-09,-125.0,1622.819946,-125.0,-125.1,-125.38,-126.18,-126.88
725051,1986-02-14,-124.0,1668.689941,-124.4,-124.97,-125.29,-126.11,-126.84
725052,1986-02-15,-124.0,1668.689941,-124.2,-124.93,-125.27,-126.09,-126.82
725054,1986-02-17,-124.0,1668.689941,-124.0,-124.87,-125.22,-126.06,-126.8
725056,1986-02-19,-124.0,1689.219971,-124.0,-124.8,-125.18,-126.03,-126.78


In [8]:
## Save predictions to an SArray
predictions = model.predict(test_data)
#predictions

### Print example predictions

In [16]:
start = 0
end = len(predictions)
step = 100

for id in range(start, end, step):
    a = round( predictions[id], 2)
    b = test_data[id]["Original"]
    print( "predicted ", round(a, 0), "\t, but actual value was \t", round(b, 0) , "\t difference is \t", round(b-a, 2) ) # dict

predicted  1585.0 	, but actual value was 	 1578.0 	 difference is 	 -7.0
predicted  2337.0 	, but actual value was 	 2319.0 	 difference is 	 -18.57
predicted  2140.0 	, but actual value was 	 2114.0 	 difference is 	 -25.89
predicted  2763.0 	, but actual value was 	 2719.0 	 difference is 	 -43.67
predicted  2963.0 	, but actual value was 	 2965.0 	 difference is 	 1.76
predicted  3253.0 	, but actual value was 	 3241.0 	 difference is 	 -12.77
predicted  3798.0 	, but actual value was 	 3833.0 	 difference is 	 34.17
predicted  4474.0 	, but actual value was 	 4494.0 	 difference is 	 19.8
predicted  6538.0 	, but actual value was 	 6509.0 	 difference is 	 -28.93
predicted  8991.0 	, but actual value was 	 9027.0 	 difference is 	 36.16
predicted  10460.0 	, but actual value was 	 10408.0 	 difference is 	 -51.72
predicted  11004.0 	, but actual value was 	 11023.0 	 difference is 	 19.07
predicted  10057.0 	, but actual value was 	 10006.0 	 difference is 	 -51.13
predicted  9150

## "Be Less Wrong"

Evaluate how good is the model

It appears that the predition results vary from run to run so it is worth to run it until you find the model with minimum error, 

or **as Elon Musk says "Be less wrong"**.

Previous results:

- {'max_error': 1749.5078773959249, 'rmse': 124.58897796835019}

TODO: write this in a loop to select the best model

### TODO: find the best model

Create a "for" loop to find the best model

In [10]:
# Evaluate the model and save the results into a dictionary
results = model.evaluate( test_data ) #test_data[0:2531]
results

{'max_error': 1749.5078773959249, 'rmse': 124.58897796835019}

## Save the model

Save the model for future use in MacOS, iOS, etc. applications

In [11]:
# Export to Core ML
model.export_coreml('./DATA/models/^DJI.mlmodel')